In [1]:
'''
GA Data Science Q2 2016

Code walk-through 9: Logistic regression using scikit-learn

* Logistic regression
* Confusion matrix and performance metrics
* Visualising the effect of covariates
* ROC analysis
* Cross-validation
* Regularisation
* Variable (feature) selection
* Multiple classes
* Stochastic gradient descent
'''

import numpy as np
import pandas as pd

from sklearn import linear_model as lm, metrics, cross_validation as cv,\
                    grid_search, feature_selection, preprocessing

from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
# Original publication: http://dx.doi.org/10.1371/journal.pone.0137524
# Dataset: http://dx.doi.org/10.5061/dryad.8jq92

#GLIOMA_URL = 'http://datadryad.org/bitstream/handle/10255/dryad.88928/An%20eighteen%20serum%20cytokine%20signature%20for%20discriminating%20glioma%20from%20normal%20healthy%20individuals%20raw%20data.xlsx?sequence=1'

glioma = pd.read_excel('../../Data/glioma.xlsx')

# Transpose DataFrame so that measurements are in columns
glioma = glioma.transpose()

# Set first row as column names, then drop it
glioma.columns = glioma.iloc[0]
glioma.columns.name = ''
glioma = glioma.reindex(glioma.index.drop('sample'))

# Extract cytokine measurements
X = glioma.iloc[:,1:].apply(pd.to_numeric, axis=1)

# Apply logarithmic transformation to each measurement
X = X.apply(np.log, axis=1)

# Dichotomise outcome: GBM versus rest
# 
# DA  = Diffuse Astrocytoma (grade II)
# AA  = Anaplastic Astrocytoma (grade III)
# GBM = Glioblastoma Multiforme (grade IV)
y = glioma.Type == 'GBM'

In [4]:

'''
Logistic regression
'''

# Fit the model
# NOTE: By default, sklearn uses L2 regularisation with parameter C (default = 1)
#       This cannot be disabled, but we can set C so big that it has little effect
model1 = lm.LogisticRegression(C=1e50)
model1.fit(X, y)

# Print regression coefficients
model1.intercept_
model1.coef_

# Print odds ratios
np.exp(model1.intercept_)
np.exp(model1.coef_)

array([[  1.37697919e+00,   1.22872574e-03,   6.23272054e+08,
          7.83649900e-03,   6.48186744e-03,   1.06837220e-02,
          3.18226502e+02,   3.08516339e-01,   7.30410085e+00,
          6.87681066e-01,   4.00884339e+08,   1.19510649e-05,
          1.00557915e-04,   2.97962454e+01,   6.18009567e-03,
          9.45412188e-01,   1.24652626e+01,   1.40437431e-04,
          1.55636052e+00,   1.39924040e-07,   1.10030198e-04,
          9.21699033e+02,   9.59616033e+03,   9.34998370e-13,
          5.37224321e+08,   2.08423766e+00,   8.53199271e-04,
          1.65402305e+04,   6.20199070e-01,   1.18056695e-01,
          1.10666811e+00,   3.51503901e-02,   8.63801257e-01,
          3.11742808e-05,   1.60621463e+02,   6.03073280e+02,
          5.34416840e-01,   3.46648739e+01,   4.19521122e-03,
          2.06086647e+04,   4.92084410e-01,   6.37174356e+05,
          6.91357679e-05,   2.69836170e-03,   7.59623292e+00,
          9.34721294e-02,   4.21655514e+01,   4.12725152e-02]])

In [6]:
'''
Confusion matrix and performance metrics
'''

# Confusion matrix
metrics.confusion_matrix(y, model1.predict(X))

# Classification accuracy
metrics.accuracy_score(y, model1.predict(X))

# Classification report
print(metrics.classification_report(y, model1.predict(X)))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00        72
       True       1.00      1.00      1.00       148

avg / total       1.00      1.00      1.00       220



In [9]:
# Repeat using L1 regularisation
kf = cv.StratifiedKFold(y, n_folds=5, shuffle=True)
model3 = lm.LogisticRegressionCV(Cs=10, cv=kf, penalty='l1', scoring='roc_auc',\
                                 solver='liblinear')
model3.fit(X, y)

LogisticRegressionCV(Cs=10, class_weight=None,
           cv=sklearn.cross_validation.StratifiedKFold(labels=[False False ..., False False], n_folds=5, shuffle=True, random_state=None),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=1, penalty='l1',
           random_state=None, refit=True, scoring='roc_auc',
           solver='liblinear', tol=0.0001, verbose=0)

In [10]:
np.exp(model3.intercept_)

array([ 1.])

In [11]:
np.exp(model3.coef_)

array([[  0.55226531,   0.86593749,  16.64530907,   0.53754231,
          0.34493249,   0.71818373,   2.63069061,   1.        ,
          1.        ,   0.82378209,  15.00369372,   0.16470895,
          0.27128577,   1.32735853,   1.        ,   1.00368011,
          1.6632326 ,   0.39535548,   1.03618749,   0.04747917,
          0.31093625,   1.11631486,   2.93800302,   0.0243767 ,
         22.52417981,   1.        ,   0.47417344,   1.09835051,
          1.04346779,   0.56432097,   1.        ,   0.66886254,
          0.88352261,   0.51324481,   1.        ,   3.04474703,
          1.        ,   1.35338467,   0.48645861,   4.89359098,
          0.63193263,   9.66814095,   0.47881418,   0.55898465,
          1.40110373,   1.        ,   2.5381217 ,   0.51013131]])

In [ ]:
plt.plot(np.log(model3.Cs_), model3.scores_[1].mean(axis=0))

In [15]:
model3.coef_[0]

array([ -5.93726721e-01,  -1.43942560e-01,   2.81212844e+00,
        -6.20747815e-01,  -1.06440657e+00,  -3.31029851e-01,
         9.67246402e-01,   0.00000000e+00,   0.00000000e+00,
        -1.93849240e-01,   2.70829642e+00,  -1.80357532e+00,
        -1.30458250e+00,   2.83190897e-01,   0.00000000e+00,
         3.67335340e-03,   5.08763056e-01,  -9.27969969e-01,
         3.55481063e-02,  -3.04746416e+00,  -1.16816736e+00,
         1.10032957e-01,   1.07773010e+00,  -3.71412735e+00,
         3.11458939e+00,   0.00000000e+00,  -7.46182123e-01,
         9.38095141e-02,   4.25495807e-02,  -5.72132100e-01,
         0.00000000e+00,  -4.02176718e-01,  -1.23838393e-01,
        -6.67002341e-01,   0.00000000e+00,   1.11341782e+00,
         0.00000000e+00,   3.02608621e-01,  -7.20603450e-01,
         1.58792639e+00,  -4.58972483e-01,   2.26883604e+00,
        -7.36442691e-01,  -5.81633273e-01,   3.37260301e-01,
         0.00000000e+00,   9.31424319e-01,  -6.73087114e-01])

In [ ]:
'''
Variable (feature) selection
'''

# Select only variables with non-zero coefficient in L1-regularised model
idx = np.where(np.abs(model3.coef_[0]) >= 1e-16)[0]

# List selected variables
X.columns[idx]

# Re-fit model using selected variables only
X_selected = X.iloc[:,idx]
model4 = lm.LogisticRegression(C=model3.C_[0])
model4.fit(X_selected, y)

# Plot ROC curve
pred_probs = model4.predict_proba(X_selected)[:,1]
fpr, tpr, cutoffs = metrics.roc_curve(y, pred_probs)
plt.plot(fpr, tpr)

# Compute area under the ROC curve (AUC)
metrics.roc_auc_score(y, pred_probs)

# Recursive Feature Elimination and Cross-Validated selection
# (using value of C found using cross-validation above)
fs = feature_selection.RFECV(lm.LogisticRegression(C=model2.C_[0]),\
                             cv=kf, scoring='roc_auc')
fs.fit(X, y)

# List selected variables
X.columns[fs.support_]

# Re-fit model using selected variables only
X_selected = X.loc[:,fs.support_]
model5 = lm.LogisticRegression(C=model2.C_[0])
model5.fit(X_selected, y)

# Plot ROC curve
pred_probs = model5.predict_proba(X_selected)[:,1]
fpr, tpr, cutoffs = metrics.roc_curve(y, pred_probs)
plt.plot(fpr, tpr)

# Compute area under the ROC curve (AUC)
metrics.roc_auc_score(y, pred_probs)

'''
Multiple classes
'''

# Check the argument `multi_class`:
# * 'ovr' means that binary models are estimated for each class
# * 'multinomial' means that a single multinomial model is estimated

# For example…
model6 = lm.LogisticRegression(C=1e50, solver='lbfgs', multi_class='multinomial')
model6.fit(X, glioma.Type)

model6.classes_
np.exp(model6.intercept_)
np.exp(model6.coef_)

'''
Stochastic gradient descent
'''

# SGD is a very efficient approach to train linear classifiers (including linear
# and logistic regression models) on large-scale and/or sparse datasets
#
# scikit-learn provides:
# * `lm.SGDRegressor` for regression problems
# * `lm.SGDClassifier` for classification problems
#
# Both support L1, L2, and Elastic Net regularisation (with parameters 'alpha'
# and 'l1_ratio' if using Elastic Net)

# SGD is sensitive to scaling of the predictors, so it’s recommended to scale
# the data to [0, 1], [-1, 1], or alternatively to standardise it to mean 0 and
# variance 1, if there’s no ‘intrinsic scale’ already

scaler = preprocessing.StandardScaler()
scaler.fit(X)

scaler.mean_
scaler.scale_

X_scaled = scaler.transform(X)

# According to the scikit-learn documentation, the following is a good guess for
# the number of iterations required to achieve convergence
n_iter = np.ceil(10**6 / X.shape[0])

# As usual, the regularisation parameter 'alpha' can be tuned using
# `grid_search.GridSearchCV`
gs = grid_search.GridSearchCV(
    estimator=lm.SGDClassifier(loss='log', penalty='l2', n_iter=n_iter),
    param_grid={'alpha': 10.0**-np.arange(1, 7)},
    scoring='roc_auc',
    cv=kf
)
gs.fit(X_scaled, y)

gs.best_estimator_

# Before using this model to predict, we'd need to call `scaler.transform` on
# the new data

# We can also put everything together in a pipeline…

sgd_pipeline = Pipeline([
    ('scale', preprocessing.StandardScaler()),
    ('sgd', lm.SGDClassifier())
])

sgd_pipeline.set_params(
    sgd__loss='log',
    sgd__penalty='l2',
    sgd__n_iter=n_iter
)

gs = grid_search.GridSearchCV(
    estimator=sgd_pipeline,
    param_grid={'sgd__alpha': 10.0**-np.arange(1, 7)},
    scoring='roc_auc',
    cv=kf
)
gs.fit(X, y)

gs.best_estimator_

# Predictions for new samples can now be obtained directly by calling
# `gs.best_estimator_.predict`



In [ ]:
'''
ROC analysis
'''

# Compute predicted probabilities for GBM (y = 1)
pred_probs = model1.predict_proba(X)[:,1]

# Confirm that model predictions assume a 50% cut-off value
assert(np.all((pred_probs >= 0.5) == model1.predict(X)))

# Visualise distribution
sns.distplot(pred_probs)

# Define a set of cut-off values where sensitivity and specificity will be computed
cutoffs = np.linspace(0, 1, 1001)

# Define a function to compute specificity
def specificity_score(y_true, y_pred):
    cm = metrics.confusion_matrix(y_true, y_pred)
    return cm[0,0] / cm[0,:].sum()

# Compute sensitivity and specificity at the cut-off values defined above
sensitivities = np.zeros(cutoffs.size)
specificities = np.zeros(cutoffs.size)
for i, cutoff in enumerate(cutoffs):
    sensitivities[i] = metrics.recall_score(y, pred_probs >= cutoff)
    specificities[i] = specificity_score(y, pred_probs >= cutoff)

# Plot the ROC curve, i.e. sensitivity versus (1 - specificity)
plt.plot(1 - specificities, sensitivities)

# Alternatively…
# (FPR = 1 - specificity; TPR = sensitivity)
fpr, tpr, cutoffs = metrics.roc_curve(y, pred_probs)
plt.plot(fpr, tpr)

# Compute area under the ROC curve (AUC)
metrics.roc_auc_score(y, pred_probs)

'''
Cross-validation
'''

# Define stratified folds
kf = cv.StratifiedKFold(y, n_folds=5, shuffle=True)

# Compute average classification accuracy across folds
accuracies = cv.cross_val_score(lm.LogisticRegression(C=1e50),\
                                X, y, scoring='accuracy', cv=kf)
np.mean(accuracies)

# Compute average AUC across folds
aucs = cv.cross_val_score(lm.LogisticRegression(C=1e50),\
                          X, y, scoring='roc_auc', cv=kf)
np.mean(aucs)